In [114]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [115]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

In [168]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -28 OUtput- 28 RF-3
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #input -28 OUtput- 28 RF-5
        
        self.pool1 = nn.MaxPool2d(2, 2) #input -28 OUtput- 14 RF-10
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input -14 OUtput- 14 RF-12
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input -14 OUtput- 14 RF-14
        self.pool2 = nn.MaxPool2d(2, 2) #input -14 OUtput- 14 RF-28

        self.conv5 = nn.Conv2d(8, 16, 3, )
        self.conv6 = nn.Conv2d(16, 32, 3, )
        
        self.batch_norm1_1 = nn.BatchNorm2d(16)
        self.batch_norm2_1 = nn.BatchNorm2d(32)
        self.batch_norm1_2 = nn.BatchNorm2d(16)
        self.batch_norm2_2 = nn.BatchNorm2d(32)
        

        self.dropout = nn.Dropout(0.1)

        self.conv1to1_32to8 = nn.Conv2d(32,8,1)
        self.conv1to1_32to10 = nn.Conv2d(32,10,1)
        
        self.gap = nn.AvgPool2d(3)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = self.batch_norm1_1(x)
        x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.batch_norm2_1(x)
        x = self.dropout(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv1to1_32to8(x))
        x = self.dropout(x)

        x = F.relu(self.conv3(x))
        x = self.batch_norm1_2(x)
        x = self.dropout(x)
        x = F.relu(self.conv4(x))
        x = self.dropout(x)
        x = self.pool2(x)

        x = F.relu(self.conv1to1_32to8(x))
        x = self.dropout(x)

        x = F.relu(self.conv5(x))
        x = self.batch_norm1_2(x)
        x = self.dropout(x)
        x = F.relu(self.conv6(x))
        x = self.dropout(x)

        
        x = self.conv1to1_32to10(x)
        
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [169]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           4,640
       BatchNorm2d-5           [-1, 32, 28, 28]              64
           Dropout-6           [-1, 32, 28, 28]               0
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             264
           Dropout-9            [-1, 8, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           1,168
      BatchNorm2d-11           [-1, 16, 14, 14]              32
          Dropout-12           [-1, 16, 14, 14]               0
           Conv2d-13           [-1, 32, 14, 14]           4,640
          Dropout-14           [-1, 32,

<ipython-input-168-4b7a73d2a1a5>:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [170]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [171]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [172]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print("Epoch",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-168-4b7a73d2a1a5>:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.041004449129104614 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.30it/s]



Test set: Average loss: 0.0821, Accuracy: 9778/10000 (98%)

Epoch 2


loss=0.03182460740208626 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.37it/s]



Test set: Average loss: 0.0747, Accuracy: 9789/10000 (98%)

Epoch 3


...


Test set: Average loss: 0.0221, Accuracy: 9926/10000 (99%)

Epoch 19


loss=0.005215884651988745 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0234, Accuracy: 9923/10000 (99%)

Epoch 20


loss=0.018454670906066895 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.08it/s]



Test set: Average loss: 0.0193, Accuracy: 9946/10000 (99%)



In [174]:
9946/10000*100

99.46000000000001